# Notebook 05: Graph Analysis

This notebook performs comprehensive network analysis on the brain connectome graph. It computes centrality measures to identify hub regions, detects community structure, and calculates global network properties. These metrics reveal the functional organization and hierarchical structure of the brain.

Key steps:
1. Load the connectome graph from Notebook 04
2. Compute centrality measures (betweenness, degree, pagerank, clustering, strength)
3. Create metrics DataFrame and identify TOP 15 HUB REGIONS
4. Save hub regions to CSV
5. Detect communities using greedy modularity optimization
6. Calculate global network properties
7. Save communities to JSON
8. Create summary visualizations

In [ ]:
# Import all required libraries
# networkx: graph algorithms and analysis
# numpy: numerical computing
# pandas: data manipulation
# json: save/load structured data
# pathlib: cross-platform file handling
# community: modularity optimization for community detection
# collections: data structures
# matplotlib: visualization
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
import networkx as nx
import community
from collections import defaultdict
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("All required libraries imported successfully")

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

print("Random seed set to 42 for reproducibility")

In [ ]:
# Load configuration and set up directories
notebook_dir = Path.cwd()
project_dir = notebook_dir.parent
data_dir = project_dir / 'data'
results_dir = project_dir / 'results'
figures_dir = project_dir / 'figures'

# Load config
config_path = data_dir / 'config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

n_regions = config['n_regions']

print(f"Configuration loaded from {config_path}")
print(f"Number of brain regions: {n_regions}")

In [ ]:
# Load the connectome graph from Notebook 04
# GraphML format preserves all node/edge properties
print("Loading connectome graph from Notebook 04...")

graph_path = results_dir / 'connectome_graph.graphml'
G = nx.read_graphml(graph_path)

# Convert node labels from strings to integers (GraphML stores as strings)
G = nx.convert_node_labels_to_integers(G)

print(f"Graph loaded from {graph_path}")
print(f"Graph properties:")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")
print(f"  Density: {nx.density(G):.4f}")

In [ ]:
# Compute betweenness centrality for all nodes
# Betweenness centrality measures how often a node lies on shortest paths between other nodes
# High betweenness = node is important for communication between different network parts
# Interpretation: identifies 'hub' regions that integrate information

print("Computing betweenness centrality...")
print("  Definition: How often a node lies on shortest paths between other nodes")
print("  Interpretation: High values indicate integration hubs")

betweenness = nx.betweenness_centrality(G, weight='weight')
betweenness = {int(k): v for k, v in betweenness.items()}

print(f"  Completed. Range: [{min(betweenness.values()):.4f}, {max(betweenness.values()):.4f}]")

In [ ]:
# Compute degree centrality for all nodes
# Degree = number of direct connections to other nodes
# Simple measure: more connections = more central

print("\nComputing degree centrality...")
print("  Definition: Number of direct connections")
print("  Interpretation: Nodes with many connections")

degree = dict(G.degree())
degree_norm = {k: v / (n_regions - 1) for k, v in degree.items()}  # Normalize by max possible

print(f"  Completed. Range: {min(degree.values())}-{max(degree.values())} connections")

In [ ]:
# Compute PageRank centrality
# Extends degree centrality by weighing connections to high-centrality nodes
# Algorithm from Google's search ranking
# High PageRank = connected to other important nodes

print("\nComputing PageRank centrality...")
print("  Definition: Weighted by importance of connected nodes")
print("  Interpretation: Identifies nodes in prominent positions")

pagerank = nx.pagerank(G, weight='weight')
pagerank = {int(k): v for k, v in pagerank.items()}

print(f"  Completed. Range: [{min(pagerank.values()):.4f}, {max(pagerank.values()):.4f}]")

In [ ]:
# Compute clustering coefficient for all nodes
# Clustering coefficient measures how connected a node's neighbors are to each other
# High clustering = node is part of a tight cluster (local specialization)
# Low clustering = node connects disparate groups (bridging function)

print("\nComputing clustering coefficient...")
print("  Definition: How tightly connected a node's neighbors are")
print("  Interpretation: High=local processing, Low=bridging between regions")

clustering = nx.clustering(G, weight='weight')
clustering = {int(k): v for k, v in clustering.items()}

print(f"  Completed. Range: [{min(clustering.values()):.4f}, {max(clustering.values()):.4f}]")

In [ ]:
# Compute weighted degree (strength) for all nodes
# Strength = sum of edge weights connected to a node
# Unlike degree (counts edges), strength considers connection intensity
# High strength = strongly connected (not just many connections)

print("\nComputing weighted degree (strength)...")
print("  Definition: Sum of weights of connected edges")
print("  Interpretation: Total functional coupling strength")

strength = {}
for node in G.nodes():
    # Sum all edge weights connected to this node
    total_weight = sum([G[node][neighbor]['weight'] for neighbor in G.neighbors(node)])
    strength[node] = total_weight

print(f"  Completed. Range: [{min(strength.values()):.4f}, {max(strength.values()):.4f}]")

In [ ]:
# Create metrics DataFrame combining all centrality measures
# Each row = one brain region
# Columns = different centrality metrics

print("\nCombining metrics into DataFrame...")

metrics_df = pd.DataFrame({
    'region_id': list(range(n_regions)),
    'degree': [degree.get(i, 0) for i in range(n_regions)],
    'betweenness': [betweenness.get(i, 0) for i in range(n_regions)],
    'pagerank': [pagerank.get(i, 0) for i in range(n_regions)],
    'clustering': [clustering.get(i, 0) for i in range(n_regions)],
    'strength': [strength.get(i, 0) for i in range(n_regions)]
})

# Normalize metrics to 0-1 range for easier comparison
# This allows combining metrics from different scales
for col in ['degree', 'betweenness', 'pagerank', 'clustering', 'strength']:
    min_val = metrics_df[col].min()
    max_val = metrics_df[col].max()
    if max_val > min_val:
        metrics_df[f'{col}_norm'] = (metrics_df[col] - min_val) / (max_val - min_val)
    else:
        metrics_df[f'{col}_norm'] = 0

# Calculate composite hub score (average of normalized metrics)
# Identifies regions that are central by multiple measures
norm_cols = [col for col in metrics_df.columns if col.endswith('_norm')]
metrics_df['hub_score'] = metrics_df[norm_cols].mean(axis=1)

# Sort by hub score to identify top hubs
metrics_df_sorted = metrics_df.sort_values('hub_score', ascending=False)

print(f"Metrics DataFrame created")
print(f"  Shape: {metrics_df.shape}")
print(f"  Columns: {list(metrics_df.columns)}")

In [ ]:
# Identify and display TOP 15 HUB REGIONS
# These are the most central regions in the brain connectome

n_top_hubs = 15
top_hubs = metrics_df_sorted.head(n_top_hubs).copy()

print("\n" + "="*100)
print("TOP 15 HUB REGIONS IN BRAIN CONNECTOME")
print("="*100)
print("\nThese regions have high centrality across multiple measures and serve as network hubs\n")

# Create formatted table
display_cols = ['region_id', 'degree', 'betweenness', 'pagerank', 'clustering', 'strength', 'hub_score']
print(f"{'Rank':<6} {'Region':<10} {'Degree':<10} {'Between':<12} {'PageRank':<12} {'Cluster':<10} {'Strength':<12} {'Hub Score':<12}")
print("-" * 100)

for rank, (idx, row) in enumerate(top_hubs.iterrows(), 1):
    print(f"{rank:<6} {int(row['region_id']):<10} {int(row['degree']):<10} "
          f"{row['betweenness']:<12.4f} {row['pagerank']:<12.4f} {row['clustering']:<10.4f} "
          f"{row['strength']:<12.4f} {row['hub_score']:<12.4f}")

print("\n" + "="*100)

In [ ]:
# Save hub regions to CSV for reference and downstream analysis
# This table can be used in other analyses or publications

hub_output_path = results_dir / 'hub_regions.csv'
top_hubs_export = top_hubs[['region_id', 'degree', 'betweenness', 'pagerank', 'clustering', 'strength', 'hub_score']].copy()
top_hubs_export['rank'] = range(1, n_top_hubs + 1)
top_hubs_export = top_hubs_export[['rank', 'region_id', 'degree', 'betweenness', 'pagerank', 'clustering', 'strength', 'hub_score']]

top_hubs_export.to_csv(hub_output_path, index=False)

print(f"\nHub regions saved to {hub_output_path}")

# Also save full metrics table
full_metrics_path = results_dir / 'all_region_metrics.csv'
metrics_export = metrics_df_sorted[['region_id', 'degree', 'betweenness', 'pagerank', 'clustering', 'strength', 'hub_score']].reset_index(drop=True)
metrics_export['rank'] = range(1, len(metrics_export) + 1)
metrics_export = metrics_export[['rank', 'region_id', 'degree', 'betweenness', 'pagerank', 'clustering', 'strength', 'hub_score']]

metrics_export.to_csv(full_metrics_path, index=False)
print(f"Full metrics saved to {full_metrics_path}")

In [ ]:
# Detect community structure using greedy modularity optimization
# Communities are groups of regions that are more densely connected to each other than to other regions
# Modularity = measure of how well-defined the communities are
# Range: -1 to 1 (1 = perfect communities, 0 = random network)

print("\nDetecting community structure...")
print("  Method: Greedy modularity optimization")
print("  Communities = densely connected subgraphs")

# Compute best partition using greedy algorithm
partition = community.best_partition(G, weight='weight')

# Convert to list of communities
communities_dict = defaultdict(list)
for node, community_id in partition.items():
    communities_dict[community_id].append(int(node))

# Calculate modularity
modularity = community.modularity(partition, G, weight='weight')

n_communities = len(communities_dict)
print(f"\nCommunity Detection Results:")
print(f"  Number of communities: {n_communities}")
print(f"  Modularity: {modularity:.4f}")
print(f"    (Ranges 0-1: higher = better communities)")

In [ ]:
# Print community information
# Shows which regions belong to which community

print("\nCommunity Composition:")
print("-" * 60)

for comm_id in sorted(communities_dict.keys()):
    regions_in_community = communities_dict[comm_id]
    n_regions_in_comm = len(regions_in_community)
    
    print(f"\nCommunity {comm_id}:")
    print(f"  Size: {n_regions_in_comm} regions")
    print(f"  Regions: {sorted(regions_in_community)}")
    
    # Calculate internal connectivity (density within community)
    if n_regions_in_comm > 1:
        subgraph = G.subgraph(regions_in_community)
        internal_edges = subgraph.number_of_edges()
        max_edges = n_regions_in_comm * (n_regions_in_comm - 1) / 2
        internal_density = internal_edges / max_edges if max_edges > 0 else 0
        print(f"  Internal density: {internal_density:.4f}")

In [ ]:
# Calculate global network properties
# These describe the overall organization of the brain connectome

print("\n" + "="*70)
print("GLOBAL NETWORK PROPERTIES")
print("="*70)

# Basic properties
print(f"\nBasic Properties:")
print(f"  Number of nodes: {G.number_of_nodes()}")
print(f"  Number of edges: {G.number_of_edges()}")

# Density
density = nx.density(G)
print(f"\nDensity:")
print(f"  {density:.4f}")
print(f"    Interpretation: {density*100:.2f}% of possible connections exist")

# Average clustering coefficient
avg_clustering = nx.average_clustering(G, weight='weight')
print(f"\nClustering:")
print(f"  Average clustering coefficient: {avg_clustering:.4f}")
print(f"    Interpretation: Measure of local cliquishness")

# Connectivity
is_connected = nx.is_connected(G)
n_components = nx.number_connected_components(G)
print(f"\nConnectivity:")
print(f"  Is connected: {is_connected}")
print(f"  Connected components: {n_components}")

# Path properties (if connected)
if is_connected:
    avg_path_length = nx.average_shortest_path_length(G, weight='weight')
    diameter = nx.diameter(G, weight='weight')
    print(f"\nPath Properties:")
    print(f"  Average shortest path: {avg_path_length:.4f}")
    print(f"  Network diameter: {diameter}")
    print(f"    Interpretation: Maximum steps needed to reach any node")

# Degree statistics
degrees = [G.degree(n) for n in G.nodes()]
print(f"\nDegree Statistics:")
print(f"  Mean degree: {np.mean(degrees):.2f}")
print(f"  Median degree: {np.median(degrees):.2f}")
print(f"  Min degree: {np.min(degrees)}")
print(f"  Max degree: {np.max(degrees)}")

# Small-world properties
print(f"\nNetwork Characteristics:")
print(f"  Modularity: {modularity:.4f}")
print(f"  Communities: {n_communities}")
print(f"\n  Brain networks typically show:")
print(f"  - High clustering (modular organization)")
print(f"  - Low average path length (efficient communication)")
print(f"  - Multiple communities (functional systems)")

In [ ]:
# Prepare community data for JSON export
# JSON is a human-readable format suitable for sharing results

communities_for_export = {
    'n_communities': n_communities,
    'modularity': float(modularity),
    'communities': {str(k): v for k, v in communities_dict.items()}
}

# Test JSON validity
try:
    json_str = json.dumps(communities_for_export)
    print("JSON structure is valid")
except Exception as e:
    print(f"Error creating JSON: {e}")

In [ ]:
# Save communities to JSON file
# This preserves the community structure for visualization and further analysis

communities_output_path = results_dir / 'communities.json'
with open(communities_output_path, 'w') as f:
    json.dump(communities_for_export, f, indent=2)

print(f"Communities saved to {communities_output_path}")

# Verify the file
file_size = communities_output_path.stat().st_size
print(f"File size: {file_size} bytes")

# Verify JSON is valid by reading it back
with open(communities_output_path, 'r') as f:
    loaded_communities = json.load(f)
print(f"JSON verification: Successfully loaded {loaded_communities['n_communities']} communities")

In [ ]:
# Create comprehensive summary and visualizations
print("\nCreating visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Brain Network Analysis Summary', fontsize=14, fontweight='bold')

# Plot 1: Top 15 hubs by hub score
ax = axes[0, 0]
top_15_for_plot = top_hubs.head(15).sort_values('hub_score')
regions = [f"R{int(r)}" for r in top_15_for_plot['region_id']]
ax.barh(regions, top_15_for_plot['hub_score'], color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Hub Score (composite centrality)')
ax.set_title('Top 15 Hub Regions')
ax.grid(True, alpha=0.3, axis='x')

# Plot 2: Centrality measures scatter plot
ax = axes[0, 1]
scatter = ax.scatter(metrics_df['degree'], metrics_df['betweenness'], 
                     c=metrics_df['hub_score'], cmap='viridis', s=50, alpha=0.6, edgecolors='black')
ax.set_xlabel('Degree (number of connections)')
ax.set_ylabel('Betweenness (pathway importance)')
ax.set_title('Degree vs Betweenness Centrality')
plt.colorbar(scatter, ax=ax, label='Hub Score')
ax.grid(True, alpha=0.3)

# Plot 3: Community sizes
ax = axes[1, 0]
community_sizes = [len(communities_dict[comm_id]) for comm_id in sorted(communities_dict.keys())]
colors_comm = plt.cm.Set3(range(n_communities))
ax.bar(sorted(communities_dict.keys()), community_sizes, color=colors_comm, alpha=0.7, edgecolor='black')
ax.set_xlabel('Community ID')
ax.set_ylabel('Number of Regions')
ax.set_title(f'Community Structure ({n_communities} communities, modularity={modularity:.3f})')
ax.grid(True, alpha=0.3, axis='y')

# Plot 4: Summary statistics
ax = axes[1, 1]
ax.axis('off')

summary_text = f"""NETWORK ANALYSIS SUMMARY

Centrality Measures (Top hub regions):
  Hub Score Range: {metrics_df['hub_score'].min():.4f}-{metrics_df['hub_score'].max():.4f}
  Degree Range: {metrics_df['degree'].min():.0f}-{metrics_df['degree'].max():.0f}
  Betweenness Range: {metrics_df['betweenness'].min():.4f}-{metrics_df['betweenness'].max():.4f}

Community Structure:
  Communities: {n_communities}
  Modularity: {modularity:.4f}
  Sizes: {sorted(community_sizes)}

Global Properties:
  Density: {density:.4f}
  Clustering: {avg_clustering:.4f}
  {f'Avg Path: {avg_path_length:.2f}' if is_connected else 'Disconnected'}

Interpretation:
  Top regions integrate information
  Communities = functional systems
  High clustering = local processing
"""

ax.text(0.05, 0.95, summary_text, transform=ax.transAxes, 
        fontsize=9, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
analysis_plot_path = figures_dir / 'graph_analysis_summary.png'
plt.savefig(analysis_plot_path, dpi=300, bbox_inches='tight')
print(f"Analysis visualization saved to {analysis_plot_path}")
plt.show()

In [ ]:
# Create final comprehensive summary
print("\n" + "="*100)
print("GRAPH ANALYSIS SUMMARY")
print("="*100)

print(f"\nCentrality Analysis:")
print(f"  Computed metrics: Degree, Betweenness, PageRank, Clustering, Strength")
print(f"  All metrics normalized to 0-1 scale")
print(f"  Composite hub score = average of normalized metrics")
print(f"  Identified {n_top_hubs} top hub regions (see hub_regions.csv)")

print(f"\nHub Regions Interpretation:")
print(f"  - High degree: Many connections (local connectivity)")
print(f"  - High betweenness: Important for information routing (integration)")
print(f"  - High pagerank: Connected to other important regions")
print(f"  - High strength: Strong functional coupling")
print(f"  - High clustering: Part of tight local modules")

print(f"\nCommunity Detection:")
print(f"  Method: Greedy modularity optimization")
print(f"  Communities found: {n_communities}")
print(f"  Modularity score: {modularity:.4f}")
print(f"  Community sizes: {sorted(community_sizes)}")

print(f"\nCommunity Interpretation:")
print(f"  - Communities = functional systems with strong internal connectivity")
print(f"  - High modularity = well-defined communities")
print(f"  - Regions in same community likely work together")

print(f"\nGlobal Network Properties:")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")
print(f"  Density: {density:.4f}")
print(f"  Clustering: {avg_clustering:.4f}")
if is_connected:
    print(f"  Avg path length: {avg_path_length:.4f}")
    print(f"  Diameter: {diameter}")

print(f"\nSmall-World Properties:")
print(f"  High clustering + low path length = efficient brain network")
print(f"  Modular organization + global integration")
print(f"  Robust to random perturbations")

print(f"\nOutput Files:")
print(f"  Top hubs: {hub_output_path}")
print(f"  All metrics: {full_metrics_path}")
print(f"  Communities: {communities_output_path}")
print(f"  Visualization: {analysis_plot_path}")

print(f"\nNext Steps:")
print(f"  1. Visualize full connectome and network structure (Notebook 06)")
print(f"  2. Create publication-quality figures")
print(f"  3. Compare with other datasets or groups")
print(f"\n" + "="*100)